In [1]:
%load_ext autoreload
%autoreload
from IPython.display import display, clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '4', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

In [28]:
print_topic_sample()

-----------Topic Samples-----------
0  bow: ! love color perfect recommend highly perfectly absolutely ordered awesome
0  sent: love love ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
1  bow: bottom top plastic speck back part piece corners feet rubber
1  sent: the only problem i have is that the bottom part of the case is not a little loose
2  bow: zipper months strap handle years broke year zippers straps back
2  sent: the only reason i did n't give it # stars is because it is a little heavy
3  bow: pocket room ipad perfect mouse power small charger perfectly carry
3  sent: there is plenty of room for the power cord , mouse , power cord , mouse , and a few other small items
4  bow: carry pockets strap room back shoulder space comfortable plenty compartment
4  sent: plenty of room for my laptop , files , and other items
5  bow: nylon sewn floor closure test waterproof empty average trust thinner
5  sent: the only reason i did n't give it # stars is becaus

## language modeling cost

In [17]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [18]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [19]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [20]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [21]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [22]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [23]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'True: %s' % true_sent)
        print(i, 'Pred: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' bow:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' sent:', pred_topic_sent)

In [24]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','LM','','VALID:','TM','','','','LM','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','NLL','KL','LOSS','PPL','NLL','KL','REG','NLL','KL', 'Beta']]))))

In [26]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, sent_loss_recon_dev, sent_loss_kl_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, '%.2f'%sent_loss_recon_train, '%.2f'%sent_loss_kl_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, '%.2f'%sent_loss_recon_dev, '%.2f'%sent_loss_kl_dev,  '%.3f'%beta_eval],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            print_sample(batch)
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                        LM        VALID:  \
      Time  Ep    Ct    LOSS   PPL     NLL    KL   REG   NLL    KL    LOSS   
1        4   0     0  141.53  1035  130.91  0.60  0.90  9.13  1.36  126.43   
501     74   0   500  120.76   575  113.80  0.27  0.41  6.22  1.36  111.16   
1001    69   0  1000  119.65   548  112.81  0.51  0.27  5.99  1.06  110.52   
1501    61   0  1500  119.20   533  112.33  0.67  0.21  5.90  0.90  110.25   
2001    61   0  2000  118.61   522  111.72  0.78  0.17  5.84  0.78  109.82   
2276    35   1     0  118.33   517  111.43  0.83  0.16  5.80  0.73  109.83   
2776    61   1   500  118.02   508  111.11  0.92  0.14  5.73  0.68  109.68   
3276    60   1  1000  117.75   502  110.84  0.99  0.12  5.67  0.63  109.29   
3776    61   1  1500  117.59   496  110.68  1.05  0.11  5.61  0.60  109.03   
4276    60   1  2000  117.40   492  110.50  1.09  0.09  5.55  0.57  109.13   
4551    34   2     0  117.39   490  110.49  1.12  0.09  5.52  0.56  108.93   
5051    60   2   500  117.27   486  110.38  1.16  0.08  5.47  0.54  109.10   
5551    61   2  1000  117.18   483  110.29  1.19  0.08  5.43  0.52  108.91   
6051    60   2  1500  117.11   480  110.22  1.22  0.07  5.39  0.51  108.72   
6551    61   2  2000  116.99   478  110.11  1.25  0.07  5.36  0.49  108.77   
6826    34   3     0  116.92   477  110.03  1.26  0.06  5.34  0.49  108.54   
7326    61   3   500  116.83   474  109.95  1.29  0.06  5.30  0.48  108.57   
7826    61   3  1000  116.77   472  109.89  1.32  0.06  5.27  0.47  108.48   
8326    61   3  1500  116.73   470  109.85  1.34  0.05  5.24  0.46  108.61   
8826    61   3  2000  116.64   469  109.76  1.36  0.05  5.22  0.45  108.28   
9101    34   4     0  116.58   467  109.70  1.38  0.05  5.20  0.45  108.27   
9601    62   4   500  116.51   465  109.63  1.40  0.05  5.18  0.44  108.44   
10101   61   4  1000  116.47   464  109.59  1.42  0.05  5.16  0.44  108.35   
10601   59   4  1500  116.39   462  109.51  1.44  0.05  5.13  0.44  108.08   
11101   58   4  2000  116.36   461  109.47  1.46  0.04  5.11  0.43  108.15   
11376   33   5     0  116.31   460  109.42  1.47  0.04  5.10  0.43  107.75   
11876   60   5   500  116.28   458  109.40  1.49  0.04  5.08  0.43  107.80   
12376   61   5  1000  116.21   457  109.34  1.51  0.04  5.06  0.44  107.61   
12876   59   5  1500  116.14   455  109.28  1.53  0.04  5.05  0.44  107.72   
13376   59   5  2000  116.07   454  109.21  1.55  0.04  5.03  0.44  107.50   
...    ...  ..   ...     ...   ...     ...   ...   ...   ...   ...     ...   
79626   33  35     0  114.35   413  107.61  2.12  0.02  4.29  0.44  106.50   
80126   59  35   500  114.34   413  107.60  2.12  0.02  4.29  0.45  106.44   
80626   59  35  1000  114.34   413  107.60  2.12  0.02  4.28  0.45  106.40   
81126   59  35  1500  114.33   413  107.60  2.12  0.02  4.28  0.45  106.49   
81626   59  35  2000  114.32   413  107.59  2.12  0.02  4.28  0.45  106.58   
81901   33  36     0  114.32   413  107.59  2.12  0.02  4.28  0.45  106.52   
82401   59  36   500  114.31   413  107.59  2.12  0.02  4.27  0.45  106.55   
82901   59  36  1000  114.31   412  107.58  2.13  0.02  4.27  0.45  106.70   
83401   59  36  1500  114.31   412  107.58  2.13  0.02  4.27  0.45  106.67   
83901   61  36  2000  114.30   412  107.58  2.13  0.02  4.27  0.45  106.81   
84176   34  37     0  114.30   412  107.58  2.13  0.02  4.26  0.45  106.79   
84676   59  37   500  114.29   412  107.57  2.13  0.02  4.26  0.45  106.79   
85176   59  37  1000  114.29   412  107.57  2.13  0.02  4.26  0.45  106.64   
85676   59  37  1500  114.29   412  107.57  2.13  0.02  4.26  0.45  106.78   
86176   59  37  2000  114.28   412  107.56  2.13  0.02  4.25  0.45  106.66   
86451   33  38     0  114.28   412  107.56  2.13  0.02  4.25  0.45  106.80   
86951   60  38   500  114.28   412  107.56  2.13  0.02  4.25  0.45  106.80   
87451   60  38  1000  114.27   412  107.55  2.14  0.02  4.25  0.45  106.84   
87951   60  38  1500  114.27   412  107.55  

0 True: i was looking for a lightweight case that i could carry the my new surface tablet in - take it on a trip , to a trade show , even just to the coffee shop or library
0 Pred: it is a for a sleeve sleeve that would could n't my ipad # asus pro , the it it out the go to the work it out
1 True: it 's great for all that
1 Pred: i 's a
2 True: the surface fits in the tablet pouch perfectly
2 Pred: the is is my the sleeve pocket with , with and , , , , , , , , , , the
3 True: room for paper pad , pens , maybe some literature if take it to a trade show , a book for on the airplane
3 Pred: the has the and , pens , etc a other
4 True: perfect for what i want
4 Pred: i bought this i needed for to
5 True: very plain , though
5 Pred: i nice with and ,
6 True: some kind of small accent or logo would be nice
6 Pred: it of of have for i
7 True: maybe a red stripe along the side with the zippered side pouch
7 Pred: the the is fit the with side pocket the power ,
8 True: so i got the shell case a

KeyboardInterrupt: 

# confirm variables

In [29]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [30]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([0.03536562, 0.00465868, 0.13373102, 0.02336785, 0.53393006,
        0.0016737 , 0.02880868, 0.18370587, 0.00324288, 0.05151561],
       dtype=float32),
 array([0.02469287, 0.00214033, 0.02091056, 0.2218967 , 0.40444553,
        0.00085018, 0.0050975 , 0.152743  , 0.01029719, 0.1569261 ],
       dtype=float32))

In [31]:
_means_topic_infer[0][:, :4]

array([[ 0.7514409 , -0.02080724, -0.08754342,  2.9888659 ],
       [ 0.7502979 ,  0.21387438, -0.21948494,  0.18009447],
       [-0.07566352,  0.01762117, -0.02816802, -0.14794706],
       [-0.24080695, -0.13825059,  0.41813016, -0.46455345],
       [-1.0908539 , -0.26050574,  0.346231  , -0.6510836 ],
       [ 0.17045614,  0.01826609, -0.01467549,  0.03143167],
       [ 0.5033383 ,  0.0995406 ,  0.01209442,  0.21555611],
       [ 0.50759584,  0.17584282, -0.22801438,  0.27870443],
       [ 2.488893  ,  0.61977255, -0.53907454,  1.2611938 ],
       [ 1.3132436 ,  0.26736495, -0.01388951,  0.16877195]],
      dtype=float32)

In [32]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [35]:
pred_means_topic

array([[ 7.52565742e-01, -1.33226626e-02, -8.45437944e-02,
         2.99679995e+00,  1.44721472e+00, -6.55502033e+00,
        -2.32751513e+00,  5.91111183e-01,  2.81779438e-01,
         2.72793269e+00, -9.56127524e-01, -1.09030294e+00,
         1.30759096e+00, -1.82090020e+00, -6.06395721e+00,
        -1.48787630e+00, -3.89525056e+00, -1.45023775e+00,
         1.95832759e-01,  1.97800130e-01, -3.61347735e-01,
         8.49354506e-01, -9.01915932e+00,  4.58554626e-01,
         1.66233563e+00,  1.80969310e+00,  1.30634201e+00,
        -1.86664283e+00, -1.29454470e+00,  8.89331937e-01,
        -2.18768215e+00, -2.42628002e+00],
       [ 7.55044699e-01,  2.25778595e-01, -2.10457146e-01,
         1.85192049e-01,  5.14446616e-01,  1.70179963e+00,
        -3.59601378e-02, -5.91997385e-01,  3.97833884e-02,
        -3.00024599e-01, -1.72570556e-01, -6.47799224e-02,
        -3.95595640e-01,  1.90289974e-01, -3.47300947e-01,
        -3.46785448e-02, -1.77649975e-01,  1.91410780e-01,
         3.10

In [ ]:
_w_mean_topic_infer[:, :10, 0]

In [ ]:
_b_means_topic

In [ ]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

In [ ]:
_enc_state_infer.shape

In [ ]:
_means_topic_infer[0]